In [1]:
import pyodbc
import pandas as pd

import warnings
warnings.filterwarnings('ignore') 

# defining the server and the database
server = '(localdb)\mssqllocaldb' 
database = 'AdventureWorksDW2012'  

# Define the connection string
conn = pyodbc.connect(
    'DRIVER={ODBC Driver 17 for SQL Server}; \
    SERVER='+ server +'; \
    DATABASE ='+ database +';\
    Trusted_Connection=yes;'
)

### 1. Get the list of products which never had an internet sale (Use EXCEPT)


In [6]:
# Thought Process:
# Query DimProduct table by EnglishProductName field 
# Implement EXCEPT 
# Join tables [AdventureWorksDW2012].[dbo].[FactInternetSales] AS Fact
# AND [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
# ON ProductKey 


data = pd.read_sql_query("""SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            
                            EXCEPT
                            
                            SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            INNER JOIN [AdventureWorksDW2012].[dbo].[FactInternetSales] AS Fact
                            ON Prod.ProductKey = Fact.ProductKey""",conn)
data

,Product Name
0,Adjustable Race
1,BB Ball Bearing
2,Bearing Ball
3,Blade
4,Cable Lock
...,...
369,Touring Rim
370,"Touring-Panniers, Large"
371,"Women's Tights, L"
372,"Women's Tights, M"


### 2. Get the list of products which never had an internet sale (Use “NOT IN”)


In [10]:
# Thought Process:
# Query DimProduct table by EnglishProductName field 
# Implement NOT IN on ProductKey in the WHERE clause
# Join tables [AdventureWorksDW2012].[dbo].[FactInternetSales] AS Fact
# AND [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
# ON ProductKey 

data = pd.read_sql_query("""SELECT DISTINCT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            
                            WHERE Prod.EnglishProductName NOT IN 
                            (SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            INNER JOIN [AdventureWorksDW2012].[dbo].[FactInternetSales] AS Fact
                            ON Prod.ProductKey = Fact.ProductKey)""",conn)
data

,Product Name
0,Adjustable Race
1,BB Ball Bearing
2,Bearing Ball
3,Blade
4,Cable Lock
...,...
369,Touring Rim
370,"Touring-Panniers, Large"
371,"Women's Tights, L"
372,"Women's Tights, M"


### 3. Combine both 1) and 2) via a UNION. The result of 3) should be the
### same result as 1) & 2)

In [11]:
data = pd.read_sql_query("""SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            
                            EXCEPT
                            
                            SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            INNER JOIN [AdventureWorksDW2012].[dbo].[FactInternetSales] AS Fact
                            ON Prod.ProductKey = Fact.ProductKey
                            
                            UNION
                            
                            SELECT DISTINCT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            
                            WHERE Prod.EnglishProductName NOT IN 
                            (SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            INNER JOIN [AdventureWorksDW2012].[dbo].[FactInternetSales] AS Fact
                            ON Prod.ProductKey = Fact.ProductKey)""",conn)
data

,Product Name
0,Adjustable Race
1,BB Ball Bearing
2,Bearing Ball
3,Blade
4,Cable Lock
...,...
369,Touring Rim
370,"Touring-Panniers, Large"
371,"Women's Tights, L"
372,"Women's Tights, M"


### 4. What happens if you replace UNION with UNION ALL in 3)


In [12]:
data = pd.read_sql_query("""SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            
                            EXCEPT
                            
                            SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            INNER JOIN [AdventureWorksDW2012].[dbo].[FactInternetSales] AS Fact
                            ON Prod.ProductKey = Fact.ProductKey
                            
                            UNION ALL
                            
                            SELECT DISTINCT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            
                            WHERE Prod.EnglishProductName NOT IN 
                            (SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            INNER JOIN [AdventureWorksDW2012].[dbo].[FactInternetSales] AS Fact
                            ON Prod.ProductKey = Fact.ProductKey)""",conn)
data

,Product Name
0,Adjustable Race
1,BB Ball Bearing
2,Bearing Ball
3,Blade
4,Cable Lock
...,...
743,Touring Rim
744,"Touring-Panniers, Large"
745,"Women's Tights, L"
746,"Women's Tights, M"


### 5. Order 4) by product name

In [13]:
# 
data = pd.read_sql_query("""SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            
                            EXCEPT
                            
                            SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            INNER JOIN [AdventureWorksDW2012].[dbo].[FactInternetSales] AS Fact
                            ON Prod.ProductKey = Fact.ProductKey
                            
                            UNION ALL
                            
                            SELECT DISTINCT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            
                            WHERE Prod.EnglishProductName NOT IN 
                            (SELECT Prod.EnglishProductName AS [Product Name]
                            FROM [AdventureWorksDW2012].[dbo].[DimProduct] AS Prod
                            INNER JOIN [AdventureWorksDW2012].[dbo].[FactInternetSales] AS Fact
                            ON Prod.ProductKey = Fact.ProductKey)
                            
                            ORDER BY 1""",conn)
data

,Product Name
0,Adjustable Race
1,Adjustable Race
2,BB Ball Bearing
3,BB Ball Bearing
4,Bearing Ball
...,...
743,"Women's Tights, L"
744,"Women's Tights, M"
745,"Women's Tights, M"
746,"Women's Tights, S"
